In [18]:
import pandas as pd
import numpy as np

In [19]:
trade_df = pd.read_csv('./trade.csv')
train_df = pd.read_csv('./user_train.csv')
test_df = pd.read_csv('./user_test.csv')

In [20]:
trade_df

,bid_id,bidder_id,auction,merchandise,device,time,country,ip,url
0,1,668d393e858e8126275433046bbd35c6tywop,aeqok,furniture,phone1,9759243157894736,in,50.201.125.84,jmqlhflrzwuay9c
1,5,e8291466de91b0eb4e1515143c7f74dexy2yr,3vi4t,mobile,phone7,9759243157894736,ru,91.107.221.27,vasstdc27m7nks3
2,9,a58ace8b671a7531c88814bc86b2a34cf0crb,3zpkj,sporting goods,phone4,9759243210526315,za,123.28.123.226,vasstdc27m7nks3
3,10,a58ace8b671a7531c88814bc86b2a34cf0crb,z6j8b,sporting goods,phone10,9759243210526315,ng,209.80.85.101,adgjz71s3p2oiau
4,13,7e34a26c4c339e875997f4b9239d2b5fscng4,nbx39,mobile,phone13,9759243263157894,in,143.183.225.33,e5i17pk9wygxewv
...,...,...,...,...,...,...,...,...,...
3071219,7656321,f5b2bbad20d1d7ded3ed960393bec0f40u6hn,gja6c,sporting goods,phone1,9709222000000000,za,231.197.103.67,pqzlexr3qrl6gmx
3071220,7656323,9655ccc7c0c193f1549475f02c54dce45kjw7,1f9t0,sporting goods,phone718,9709222000000000,us,199.76.204.89,vasstdc27m7nks3
3071221,7656325,93e3c34171ab65e0b031dc67461e48b478u36,jmny0,jewelry,phone892,9709222052631578,id,136.144.47.247,36phop3q8vduhf5
3071222,7656326,03aafab9868455b78f0723eda86698ba9v9rm,lw44y,mobile,phone16,9709222052631578,in,5.146.235.202,oqknoadbcbcm6so


In [21]:
def trade_data_processing(trade_df):
    # calculate the number of trades for each user计算每个用户的交易数量
    trade_count = trade_df.groupby('bidder_id').size().reset_index(name='trade_count')
    trade_count = trade_count.reset_index(drop=True)

    # get the most used device for each user计算每个用户最常用的设备
    device_counts = trade_df.groupby(['bidder_id', 'device']).size()
    device_counts = device_counts.reset_index(name='counts')
    most_used_devices = device_counts.loc[device_counts.groupby('bidder_id')['counts'].idxmax()]
    most_used_devices = most_used_devices.rename(columns={"device": "most_used_device"}).reset_index(drop=True)

    # get the most used country for each user计算每个用户最常在的国家
    country_counts = trade_df.groupby(['bidder_id', 'country']).size()
    country_counts = country_counts.reset_index(name='counts')
    most_used_countries = country_counts.loc[country_counts.groupby('bidder_id')['counts'].idxmax()]
    most_used_countries = most_used_countries.rename(columns={"country": "most_used_country"}).reset_index(drop=True)

    # merge all the dataframes合并所有表格
    feature_df = trade_count.merge(most_used_devices, on='bidder_id', how='left').reset_index(drop=True)
    feature_df = feature_df.merge(most_used_countries, on='bidder_id', how='left').reset_index(drop=True)
    return feature_df



In [22]:
# 输出上面的结果
user_feature_df = trade_data_processing(trade_df)
user_feature_df

,bidder_id,trade_count,most_used_device,counts_x,most_used_country,counts_y
0,001068c415025a009fee375a12cff4fcnht8y,1,phone561,1,bn,1.0
1,0030a2dd87ad2733e0873062e4f83954mkj86,1,phone313,1,ir,1.0
2,00a0517965f18610417ee784a05f494d4dw6e,141,phone3,10,in,83.0
3,00cc97158e6f4cb8eac3c0075918b7ffi5k8o,3,phone106,1,id,2.0
4,01067975436d123f717ee5aba0dd4bbfa0937,543,phone19,29,in,123.0
...,...,...,...,...,...,...
1979,ff92ea4abd33ed38601287f0e1d6726dmgx1f,395,phone4,137,lk,395.0
1980,ffacbed056cbfaa60c1fcf51f0d381bddr3ly,9,phone150,2,my,6.0
1981,ffaf0a972a6dcb3910fd6b16045781e2ava5y,3,phone8,2,de,1.0
1982,ffc4e2dd2cc08249f299cab46ecbfacfobmr3,22,phone65,4,ng,12.0


In [23]:
def one_hot_encode(df, column_name):
    one_hot = pd.get_dummies(df[column_name], prefix=column_name)
    df = df.drop(column_name, axis=1)
    df = df.join(one_hot)
    return df

user_feature_df = one_hot_encode(user_feature_df, 'most_used_device')
user_feature_df = one_hot_encode(user_feature_df, 'most_used_country')
user_feature_df

,bidder_id,trade_count,counts_x,counts_y,most_used_device_phone0,most_used_device_phone1,most_used_device_phone1003,most_used_device_phone1004,most_used_device_phone1008,most_used_device_phone101,...,most_used_country_tr,most_used_country_tz,most_used_country_ua,most_used_country_uk,most_used_country_us,most_used_country_vn,most_used_country_ye,most_used_country_za,most_used_country_zm,most_used_country_zw
0,001068c415025a009fee375a12cff4fcnht8y,1,1,1.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,0030a2dd87ad2733e0873062e4f83954mkj86,1,1,1.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,00a0517965f18610417ee784a05f494d4dw6e,141,10,83.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,00cc97158e6f4cb8eac3c0075918b7ffi5k8o,3,1,2.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,01067975436d123f717ee5aba0dd4bbfa0937,543,29,123.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1979,ff92ea4abd33ed38601287f0e1d6726dmgx1f,395,137,395.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1980,ffacbed056cbfaa60c1fcf51f0d381bddr3ly,9,2,6.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1981,ffaf0a972a6dcb3910fd6b16045781e2ava5y,3,2,1.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1982,ffc4e2dd2cc08249f299cab46ecbfacfobmr3,22,4,12.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [24]:
# merge feature_df with train_df and test_df
train_df = train_df.merge(user_feature_df, left_on='bidder_id', right_on='bidder_id', how='left')
test_df = test_df.merge(user_feature_df, left_on='bidder_id', right_on='bidder_id', how='left')
train_df

,bidder_id,payment_account,address,outcome,trade_count,counts_x,counts_y,most_used_device_phone0,most_used_device_phone1,most_used_device_phone1003,...,most_used_country_tr,most_used_country_tz,most_used_country_ua,most_used_country_uk,most_used_country_us,most_used_country_vn,most_used_country_ye,most_used_country_za,most_used_country_zm,most_used_country_zw
0,aee383561c0019dc01552bfa5263af8etbuc7,296fa05acf2f73c8438562940d2b8204uhxbp,c0649afaeb5df1acbb856a732cd76424k9rq8,0.0,54.0,24.0,24.0,False,False,False,...,False,False,False,True,False,False,False,False,False,False
1,ec6d3fbd8408ba43891370d4ffe41eabhnwbs,a3d2de7675556553a5f08e4c88d2c228r5zgb,8e67687228a28858db3b899574989785c3yxw,0.0,10.0,10.0,8.0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,b5d61accc925a0e86ab7274e1ad82532gb1w6,a3d2de7675556553a5f08e4c88d2c228z3tuh,a3d2de7675556553a5f08e4c88d2c228b83tj,0.0,1.0,1.0,1.0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,683833b860c003e6c31c521c9cdb49a1kr6b4,a3d2de7675556553a5f08e4c88d2c228jg51s,a3d2de7675556553a5f08e4c88d2c228i3bb3,0.0,1.0,1.0,1.0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,bc7722baf876d0735ddc885691eb687bgz365,a3d2de7675556553a5f08e4c88d2c228anxa0,d20c4c76e701af07f7d77db657e7ba70cq4w1,0.0,519.0,64.0,415.0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,74866aaa89db79f25dc0fae8be067b83xlbm6,a3d2de7675556553a5f08e4c88d2c228ww56s,a3d2de7675556553a5f08e4c88d2c228v0mbb,0.0,12.0,3.0,11.0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
499,62ac0de5f8776ca1d189297bdea9a64cmewb9,dc810553cbe65ed2115a6c3bd3545c4cphlcz,846c31cc9b3e534fb66d758fb7305deee4ge5,0.0,2.0,1.0,2.0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
500,c780bc2dc7ab83d5e5c85dd13f0f7fdecfxjq,a165c844ab870a708c54ca6e669f14d9c5pep,c98f3e2b040d17a891e74410dce9e13amfceu,1.0,1317.0,113.0,1189.0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
501,4dd7f98cfffffd6f4707fd0deb4a00b5rj9wv,802b586a373333f0d1737c049782aeaeqbvdy,ca94320962ecec2fd1549f647fa2b4ad93zas,0.0,19844.0,978.0,8175.0,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [34]:
print(train_df.dtypes)


bidder_id                object
payment_account          object
address                  object
outcome                 float64
trade_count             float64
                         ...   
most_used_country_vn     object
most_used_country_ye     object
most_used_country_za     object
most_used_country_zm     object
most_used_country_zw     object
Length: 410, dtype: object


In [25]:
# change all bool value to float,但是输出看着还是bool形式
for column in train_df.columns:
    if train_df[column].dtype == bool:
        train_df[column] = train_df[column].astype(np.float64)
for column in test_df.columns:
    if test_df[column].dtype == bool:
        test_df[column] = test_df[column].astype(np.float64)

train_df

,bidder_id,payment_account,address,outcome,trade_count,counts_x,counts_y,most_used_device_phone0,most_used_device_phone1,most_used_device_phone1003,...,most_used_country_tr,most_used_country_tz,most_used_country_ua,most_used_country_uk,most_used_country_us,most_used_country_vn,most_used_country_ye,most_used_country_za,most_used_country_zm,most_used_country_zw
0,aee383561c0019dc01552bfa5263af8etbuc7,296fa05acf2f73c8438562940d2b8204uhxbp,c0649afaeb5df1acbb856a732cd76424k9rq8,0.0,54.0,24.0,24.0,False,False,False,...,False,False,False,True,False,False,False,False,False,False
1,ec6d3fbd8408ba43891370d4ffe41eabhnwbs,a3d2de7675556553a5f08e4c88d2c228r5zgb,8e67687228a28858db3b899574989785c3yxw,0.0,10.0,10.0,8.0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,b5d61accc925a0e86ab7274e1ad82532gb1w6,a3d2de7675556553a5f08e4c88d2c228z3tuh,a3d2de7675556553a5f08e4c88d2c228b83tj,0.0,1.0,1.0,1.0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,683833b860c003e6c31c521c9cdb49a1kr6b4,a3d2de7675556553a5f08e4c88d2c228jg51s,a3d2de7675556553a5f08e4c88d2c228i3bb3,0.0,1.0,1.0,1.0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,bc7722baf876d0735ddc885691eb687bgz365,a3d2de7675556553a5f08e4c88d2c228anxa0,d20c4c76e701af07f7d77db657e7ba70cq4w1,0.0,519.0,64.0,415.0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,74866aaa89db79f25dc0fae8be067b83xlbm6,a3d2de7675556553a5f08e4c88d2c228ww56s,a3d2de7675556553a5f08e4c88d2c228v0mbb,0.0,12.0,3.0,11.0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
499,62ac0de5f8776ca1d189297bdea9a64cmewb9,dc810553cbe65ed2115a6c3bd3545c4cphlcz,846c31cc9b3e534fb66d758fb7305deee4ge5,0.0,2.0,1.0,2.0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
500,c780bc2dc7ab83d5e5c85dd13f0f7fdecfxjq,a165c844ab870a708c54ca6e669f14d9c5pep,c98f3e2b040d17a891e74410dce9e13amfceu,1.0,1317.0,113.0,1189.0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
501,4dd7f98cfffffd6f4707fd0deb4a00b5rj9wv,802b586a373333f0d1737c049782aeaeqbvdy,ca94320962ecec2fd1549f647fa2b4ad93zas,0.0,19844.0,978.0,8175.0,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [26]:
# fill all the NaN values with 0
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)

In [27]:
# split the data into train and valid
from sklearn.model_selection import train_test_split
train_df, valid_df = train_test_split(train_df, test_size=0.75, random_state=42)

In [28]:
# prepare the training data
y_train = train_df['outcome']
X_train = train_df.drop(columns=['outcome']).select_dtypes(include=["number", "float"])

# use a simple logistic regression model
from sklearn.tree import DecisionTreeClassifier # 决策树
from sklearn.neighbors import KNeighborsClassifier # KNN
from sklearn.linear_model import LogisticRegression
model = KNeighborsClassifier()
model.fit(X_train, y_train)



KNeighborsClassifier()

In [29]:
X_train

,trade_count,counts_x,counts_y
432,25.0,2.0,7.0
138,12901.0,877.0,3883.0
62,5.0,3.0,4.0
135,7.0,5.0,5.0
128,123.0,21.0,39.0
...,...,...,...
106,200.0,152.0,156.0
270,41.0,6.0,41.0
348,14.0,3.0,9.0
435,1.0,1.0,1.0


In [30]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=50)
clf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=50)

In [31]:
X_train.dtypes

trade_count    float64
counts_x       float64
counts_y       float64
dtype: object

In [32]:
# convert id to index
y_valid = valid_df['outcome']
X_valid = valid_df.drop(columns=['outcome']).select_dtypes(include=["number", "bool"])

# predict the validation data
y_pred = model.predict_proba(X_valid)[:, 1]
y_pred1 = clf.predict_proba(X_valid)[:, 1]

# calculate the auc
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(y_valid, y_pred)

auc1 = roc_auc_score(y_valid, y_pred1)
print("Valid aUC: ", auc)
print("Valid aUC: ", auc1)


Valid aUC:  0.662849162011173
Valid aUC:  0.7120810055865922


In [33]:
# predict the test data
X_test = test_df.select_dtypes(include=["number", "bool"])
y_test = clf.predict_proba(X_test)[:, 1]

# save the result
test_df['prediction'] = y_test
test_df[['bidder_id', 'prediction']].to_csv('result.csv', index=False)